# Weekly Aggregation of Weather Data

**Methodology:**
1. 



In [ ]:
import pandas as pd
print(pd.__version__)

2.2.3


In [2]:
#import weather and date files and convert to dataframe
weather_extracted = pd.read_csv('weather_data_extracted.csv')
unique_env_dates = pd.read_csv('final_unique_combinations_overlap_weather_data.csv')

In [5]:
unique_env_dates.dtypes, weather_extracted.dtypes

(Env               object
 Date_Planted      object
 Date_Harvested    object
 dtype: object,
 Env                   object
 Date                   int64
 T2MDEW               float64
 RH2M                 float64
 WS2M                 float64
 ALLSKY_SFC_SW_DWN    float64
 T2M_MAX              float64
 T2M_MIN              float64
 PRECTOTCORR          float64
 dtype: object)

In [27]:
#convert 'Date' objects to datetime 
unique_env_dates['Date_Planted'] = pd.to_datetime(unique_env_dates['Date_Planted'])
unique_env_dates['Date_Harvested'] = pd.to_datetime(unique_env_dates['Date_Harvested'])
print(unique_env_dates.dtypes)

Env                       object
Date_Planted      datetime64[ns]
Date_Harvested    datetime64[ns]
dtype: object


In [3]:
#convert 'Date' int64 to datetime 
weather_extracted['Date'] = pd.to_datetime(weather_extracted['Date'], format='%Y%m%d')
print(weather_extracted.dtypes)

Env                          object
Date                 datetime64[ns]
T2MDEW                      float64
RH2M                        float64
WS2M                        float64
ALLSKY_SFC_SW_DWN           float64
T2M_MAX                     float64
T2M_MIN                     float64
PRECTOTCORR                 float64
dtype: object


In [29]:
unique_env_dates.head(5)

,Env,Date_Planted,Date_Harvested
0,DEH1_2014,2014-05-05,2014-09-29
1,GAH1_2014,2014-04-04,2014-09-11
2,IAH1a_2014,2014-05-17,2014-10-21
3,IAH1b_2014,2014-05-17,2014-10-20
4,IAH1c_2014,2014-05-09,2014-10-20


In [30]:
len(unique_env_dates)
#413 unique combinations of Env, Date_Planted and Date_Harvested (see Weather_data_exploration.ipynb in g2fproject/Processing)

413

In [9]:
weather_extracted.head(5)

,Env,Date,T2MDEW,RH2M,WS2M,ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_MIN,PRECTOTCORR
0,DEH1_2015,2015-01-01,-3.86,74.81,3.13,10.75,6.00,-3.39,0.00
1,DEH1_2015,2015-01-02,-1.08,79.81,2.02,5.72,8.23,-1.47,0.00
2,DEH1_2015,2015-01-03,3.57,92.81,1.90,1.74,7.77,-1.35,6.70
3,DEH1_2015,2015-01-04,12.58,95.00,5.38,4.05,18.62,6.78,9.98
4,DEH1_2015,2015-01-05,-4.08,63.38,5.17,10.92,8.73,-3.14,0.00


### Weekly aggregation, padding and wide to long formatting for 413 environments in weather data file.
#### 9/April/2025
- Aggregate weather variables by week relative to Date_Planted
- Due to variable growth cycles, applying padding to capture weather patterns that extend beyond typical crop cycles
- Melt and transpose wide format data to long format data for Machine learning

In [31]:
from datetime import timedelta

# Create an empty dictionary to store weather data for each unique (Env, Plant, Harvest) combo
weather_by_instance = {}

# Loop through each row in the practice_envs dataframe
for _, row in unique_env_dates.iterrows():
    env = row['Env']
    plant_date = pd.to_datetime(row['Date_Planted'])
    harvest_date = pd.to_datetime(row['Date_Harvested'])
    start_date = plant_date - timedelta(weeks=10)   #start date is 10 week before Date_Planted

    # Filter the weather_data for the specific environment and date range
    filtered_weather = weather_extracted[
        (weather_extracted['Env'] == env) &
        (weather_extracted['Date'] >= start_date) &
        (weather_extracted['Date'] < harvest_date)
    ].copy()

    # Store filtered weather data as dataframe in the weather_by_instance dictionary with unique env-plantdate-harvestdate key
    key = (env, plant_date.strftime('%Y-%m-%d'), harvest_date.strftime('%Y-%m-%d'))
    weather_by_instance[key] = filtered_weather

In [32]:
len(weather_by_instance)
#413 dataframes in dictionary
#one for each combo

413

In [33]:
#add column for 'Week' (relative to Date_Planted)
for key, df in weather_by_instance.items():
    env, plant_str, harvest_str = key
    plant_date = pd.to_datetime(plant_str)
    
    # Calculate week number relative to Date_Planted
    df['Week'] = ((df['Date'] - plant_date).dt.days // 7)
    

In [34]:
len(weather_by_instance)

413

In [16]:
#check first dataframe in weather_by_instance dict
weather_by_instance['DEH1_2014','2014-05-05','2014-09-29']

,Env,Date,T2MDEW,RH2M,WS2M,ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_MIN,PRECTOTCORR,Week
80396,DEH1_2014,2014-02-24,-2.92,72.94,4.32,17.72,6.23,-3.49,0.02,-10
80397,DEH1_2014,2014-02-25,-3.88,82.75,1.69,6.19,1.75,-3.58,0.43,-10
80398,DEH1_2014,2014-02-26,-5.02,79.38,2.38,8.63,1.83,-5.01,0.61,-10
80399,DEH1_2014,2014-02-27,-7.19,66.94,4.30,15.44,3.73,-6.10,0.00,-10
80400,DEH1_2014,2014-02-28,-12.68,63.38,2.82,18.94,-2.44,-10.71,0.00,-10
...,...,...,...,...,...,...,...,...,...,...
80608,DEH1_2014,2014-09-24,14.51,77.94,4.87,7.27,21.91,13.26,10.35,20
80609,DEH1_2014,2014-09-25,18.26,88.12,4.33,7.38,26.22,15.69,21.97,20
80610,DEH1_2014,2014-09-26,13.56,76.56,2.38,19.69,24.92,13.73,0.10,20
80611,DEH1_2014,2014-09-27,14.05,76.12,1.68,20.33,27.16,12.33,0.00,20


In [35]:
#Weekly aggregation of weather variables

aggregated_weather_data = {}

for key, df in weather_by_instance.items():
    env, plant_str, harvest_str = key  # Extract env and dates
    plant_date = pd.to_datetime(plant_str)

    # Perform weekly aggregation
    aggregated_df = df.groupby('Week').agg(
        T2MDEW=('T2MDEW', 'sum'),
        RH2M=('RH2M', 'mean'),
        WS2M=('WS2M', 'sum'),
        ALLSKY_SFC_SW_DWN=('ALLSKY_SFC_SW_DWN', 'sum'),
        T2M_MAX=('T2M_MAX', 'mean'),
        T2M_DIFF_SUM=('T2M_MAX', lambda x: (x - df.loc[x.index, 'T2M_MIN']).sum()),
        T2M_GDD=('T2M_MAX', lambda x: (((x + df.loc[x.index, 'T2M_MIN']) / 2) - 10).sum()),
        T2M_MIN=('T2M_MIN', 'mean'),
        PRECTOTCORR=('PRECTOTCORR', 'sum')
    ).reset_index()

    # Store the result in aggregated_weather_data dictionary
    aggregated_weather_data[key] = aggregated_df

In [36]:
len(aggregated_weather_data)
#413 instances

413

## Weather Features
- **ALLSKY_SFC_SW_DWN** = Sum of **All Sky Surface Shortwave Downward Irradiance** or solar radiation which reaches the Earth's surface.  
- **PRECTOTCORR** = Sum of **Precipitation corrected** or total rainfall.  
- **RH2M** = Mean **Relative Humidity at 2 Meters**.  
- **T2MDEW** = Sum of **Dew/Frost Point at 2 Meters** (negative possible).  
- **T2M_DIFF_SUM** = Sum of the difference between **max** and **min temperatures**.  
- **T2M_GDD** = Total **Growing Degree Days**, which indicates how much heat can contribute to growth (negative possible = no growth).  
- **T2M_MAX** = Mean **Max Temperature** (negative possible).  
- **T2M_MIN** = Mean **Min Temperature** (negative possible).  
- **WS2M** = Sum of **Wind Speed at 2 Meters**.



In [18]:
#check first dataframe in aggregated_weather_data dict
aggregated_weather_data['DEH1_2014','2014-05-05','2014-09-29']

,Week,T2MDEW,RH2M,WS2M,ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_DIFF_SUM,T2M_GDD,T2M_MIN,PRECTOTCORR
0,-10,-34.53,76.304286,19.50,93.01,4.051429,66.62,-74.950,-5.465714,4.55
1,-9,-18.68,83.651429,26.65,79.59,4.697143,60.18,-67.210,-3.900000,11.54
2,-8,9.61,76.160000,30.43,103.42,11.221429,78.99,-30.945,-0.062857,11.20
3,-7,16.60,82.962857,26.39,83.33,10.072857,67.24,-33.110,0.467143,9.91
4,-6,7.40,81.464286,29.90,91.60,8.432857,61.74,-41.840,-0.387143,48.70
5,-5,38.32,80.357143,21.17,134.76,15.347143,83.88,-4.510,3.364286,1.15
6,-4,68.14,81.552857,25.06,132.94,18.721429,81.40,20.350,7.092857,12.52
7,-3,40.38,80.285714,28.81,134.92,14.900000,78.80,-5.100,3.642857,29.99
8,-2,49.60,73.564286,25.87,180.16,17.878571,84.30,13.000,5.835714,15.99
9,-1,80.37,81.902857,26.96,100.98,19.217143,60.17,34.435,10.621429,54.64


In [ ]:
# padding
import numpy as np

# Define the full range of weeks (-10 to 31)
full_weeks = np.arange(-10, 32)  

# Loop through each dataframe in the dictionary
for key, df in aggregated_weather_data.items():
    
    # Create a dataframe with all weeks
    full_week_df = pd.DataFrame({'Week': full_weeks})
    
    # Merge with the existing dataframe on 'Week' column
    padded_df = full_week_df.merge(df, on='Week', how='left')
    
    # Fill missing values with 0
    padded_df.fillna(0, inplace=True)
    
    # Store the updated dataframe back in the aggregated_weather_data dictionary
    aggregated_weather_data[key] = padded_df

In [38]:
len(aggregated_weather_data)
#413 instances

413

In [20]:
#check first dataframe in aggregated_weather_data dict with padding
aggregated_weather_data['DEH1_2014','2014-05-05','2014-09-29'].tail(12)

,Week,T2MDEW,RH2M,WS2M,ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_DIFF_SUM,T2M_GDD,T2M_MIN,PRECTOTCORR
30,20,94.95,74.427143,20.41,108.28,24.692857,78.67,63.515,13.454286,32.43
31,21,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000,0.000000,0.00
32,22,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000,0.000000,0.00
33,23,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000,0.000000,0.00
34,24,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000,0.000000,0.00
35,25,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000,0.000000,0.00
36,26,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000,0.000000,0.00
37,27,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000,0.000000,0.00
38,28,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000,0.000000,0.00
39,29,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000,0.000000,0.00


In [20]:
def transpose_and_combine_data(filtered_weather_data):
    """
    Transpose and combine weekly weather data for unique environment, Date_Planted, Date_Harvested combinations.

    For each (Environment, Date_Planted, Date_Harvested) group, this function reshapes 
    the weather data from wide to long format, then pivots it so each weather variable 
    per week becomes a separate column. The final output is a single DataFrame combining 
    all groups.

    Parameters:
        filtered_weather_data (dictionary): 
            Dictionary with keys as (Environment, Date_Planted, Date_Harvested) and values 
            as DataFrames with weekly weather data.

    Returns:
        pd.DataFrame: Combined DataFrame with transposed weather variables per group.
    """
    transposed_data_list = []

    for key, data in filtered_weather_data.items():
        env, date_planted, date_harvested = key
        
        #melt filtered and padded weather data
        melted_data = data.melt(id_vars=['Week'], var_name='Variable', value_name='Value') #melt data from wide to long format
        melted_data['Env'] = env #add Env col
        melted_data['Date_Planted'] = date_planted #add Date_Planted col
        melted_data['Date_Harvested'] = date_harvested #add #Date_Harvested col
        melted_data['Week_Variable'] = melted_data['Variable'] + " - Week " + melted_data['Week'].astype(str) #create Week_variable combined col 

        #transpose melted data
        transposed_data = melted_data.pivot(
            index=['Env', 'Date_Planted', 'Date_Harvested'], # Index columns to ensure correct grouping
            columns='Week_Variable',  # Each unique Week-Variable combination becomes a new column
            values='Value' # The weather data values for each week-variant are placed in the columns
        )  
        transposed_data.reset_index(inplace=True)
        transposed_data_list.append(transposed_data) #add to list

    final_transposed_df = pd.concat(transposed_data_list, ignore_index=True) #concatenated into list into df

    return final_transposed_df

In [43]:
transposed_combined_agggregated_weather_data = transpose_and_combine_data(aggregated_weather_data)

In [44]:
transposed_combined_agggregated_weather_data.head(5) 
#weeks not in order

Week_Variable,Env,Date_Planted,Date_Harvested,ALLSKY_SFC_SW_DWN - Week -1,ALLSKY_SFC_SW_DWN - Week -10,ALLSKY_SFC_SW_DWN - Week -2,ALLSKY_SFC_SW_DWN - Week -3,ALLSKY_SFC_SW_DWN - Week -4,ALLSKY_SFC_SW_DWN - Week -5,ALLSKY_SFC_SW_DWN - Week -6,...,WS2M - Week 29,WS2M - Week 3,WS2M - Week 30,WS2M - Week 31,WS2M - Week 4,WS2M - Week 5,WS2M - Week 6,WS2M - Week 7,WS2M - Week 8,WS2M - Week 9
0,DEH1_2014,2014-05-05,2014-09-29,100.98,93.01,180.16,134.92,132.94,134.76,91.60,...,0.0,18.04,0.0,0.0,16.96,19.27,13.87,17.95,24.49,21.36
1,GAH1_2014,2014-04-04,2014-09-11,153.08,67.50,146.46,98.51,127.10,88.60,94.72,...,0.0,19.35,0.0,0.0,13.45,14.72,13.97,10.57,14.34,15.06
2,IAH1a_2014,2014-05-17,2014-10-21,109.78,114.57,154.81,88.07,120.98,130.39,140.41,...,0.0,18.02,0.0,0.0,25.20,14.26,22.45,18.51,14.86,20.32
3,IAH1b_2014,2014-05-17,2014-10-20,109.78,114.57,154.81,88.07,120.98,130.39,140.41,...,0.0,18.02,0.0,0.0,25.20,14.26,22.45,18.51,14.86,20.32
4,IAH1c_2014,2014-05-09,2014-10-20,155.14,86.00,92.67,120.51,128.77,126.89,101.45,...,0.0,20.56,0.0,0.0,17.41,25.55,12.43,23.22,18.31,15.53


In [45]:
len(transposed_combined_agggregated_weather_data)

413

In [46]:
#order column names by week
import re

# Metadata columns 
metadata_cols = ['Env', 'Date_Planted', 'Date_Harvested']

# All other columns (the week-variable columns)
week_cols = [col for col in transposed_combined_agggregated_weather_data.columns if col not in metadata_cols]

# Function to extract numeric week from column name
def extract_week_number(col_name):
    match = re.search(r'Week (-?\d+)', col_name)
    return int(match.group(1)) if match else float('inf')

# Sort week columns by extracted week number
sorted_week_cols = sorted(week_cols, key=extract_week_number)

# Reorder DataFrame
transposed_combined_agggregated_weather_data = transposed_combined_agggregated_weather_data[metadata_cols + sorted_week_cols]

In [47]:
transposed_combined_agggregated_weather_data.head(5)

Week_Variable,Env,Date_Planted,Date_Harvested,ALLSKY_SFC_SW_DWN - Week -10,PRECTOTCORR - Week -10,RH2M - Week -10,T2MDEW - Week -10,T2M_DIFF_SUM - Week -10,T2M_GDD - Week -10,T2M_MAX - Week -10,...,WS2M - Week 30,ALLSKY_SFC_SW_DWN - Week 31,PRECTOTCORR - Week 31,RH2M - Week 31,T2MDEW - Week 31,T2M_DIFF_SUM - Week 31,T2M_GDD - Week 31,T2M_MAX - Week 31,T2M_MIN - Week 31,WS2M - Week 31
0,DEH1_2014,2014-05-05,2014-09-29,93.01,4.55,76.304286,-34.53,66.62,-74.950,4.051429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,GAH1_2014,2014-04-04,2014-09-11,67.50,18.81,80.685714,-5.32,73.49,-51.765,7.854286,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,IAH1a_2014,2014-05-17,2014-10-21,114.57,0.12,69.045714,-17.18,99.22,-43.620,10.855714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,IAH1b_2014,2014-05-17,2014-10-20,114.57,0.12,69.045714,-17.18,99.22,-43.620,10.855714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,IAH1c_2014,2014-05-09,2014-10-20,86.00,6.79,81.088571,-95.50,86.73,-146.445,-4.725714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
transposed_combined_agggregated_weather_data.to_csv('transposed_combined_agggregated_weather_data.csv', index=False)
#need to normalise environment variables with loops 
#must mask out zero values?? 
#add life cycle of plant in weeks

### Normalise Data

In [ ]:
from sklearn.preprocessing import MinMaxScaler
#normalise weather variables

# Step 1: Keep first 3 columns as-is
aggregate_weather_data_normalized = transposed_combined_agggregated_weather_data[['Env', 'Date_Planted', 'Date_Harvested']].copy()

# Step 2: Select columns to normalize
columns_to_normalize = transposed_combined_agggregated_weather_data.columns.difference(['Env', 'Date_Planted', 'Date_Harvested'])

# Step 3: Normalize only non-zero values
scaler = MinMaxScaler()

for col in columns_to_normalize:
    values = transposed_combined_agggregated_weather_data[col].values.astype(float)
    mask = values != 0  # mask out non-zero values

    if np.any(mask):  # only fit if there are non-zero values
        #reshape for MinMax package
        scaled = scaler.fit_transform(values[mask].reshape(-1, 1)).flatten()
        normalized_col = values.copy()
        normalized_col[mask] = scaled
        aggregate_weather_data_normalized[col] = normalized_col
    else:
        aggregate_weather_data_normalized[col] = values  # keep as is if all zero


In [41]:
aggregate_weather_data_normalized.head(5)

,Env,Date_Planted,Date_Harvested,ALLSKY_SFC_SW_DWN - Week -1,ALLSKY_SFC_SW_DWN - Week -10,ALLSKY_SFC_SW_DWN - Week -2,ALLSKY_SFC_SW_DWN - Week -3,ALLSKY_SFC_SW_DWN - Week -4,ALLSKY_SFC_SW_DWN - Week -5,ALLSKY_SFC_SW_DWN - Week -6,...,WS2M - Week 29,WS2M - Week 3,WS2M - Week 30,WS2M - Week 31,WS2M - Week 4,WS2M - Week 5,WS2M - Week 6,WS2M - Week 7,WS2M - Week 8,WS2M - Week 9
0,DEH1_2014,2014-05-05,2014-09-29,0.264668,0.624749,0.960744,0.625711,0.607370,0.655227,0.341976,...,0.0,0.468576,0.0,0.0,0.438078,0.476419,0.513678,0.596473,0.908885,0.669540
1,GAH1_2014,2014-04-04,2014-09-11,0.678884,0.441750,0.703865,0.349731,0.567884,0.291045,0.366714,...,0.0,0.503315,0.0,0.0,0.345392,0.361665,0.517477,0.346219,0.525142,0.468391
2,IAH1a_2014,2014-05-17,2014-10-21,0.334632,0.779412,0.767513,0.270598,0.526504,0.620750,0.728988,...,0.0,0.468046,0.0,0.0,0.655664,0.350063,0.839666,0.615463,0.544802,0.636335
3,IAH1b_2014,2014-05-17,2014-10-20,0.334632,0.779412,0.767513,0.270598,0.526504,0.620750,0.728988,...,0.0,0.468046,0.0,0.0,0.655664,0.350063,0.839666,0.615463,0.544802,0.636335
4,IAH1c_2014,2014-05-09,2014-10-20,0.695262,0.574462,0.293849,0.516486,0.579175,0.593136,0.420076,...,0.0,0.535402,0.0,0.0,0.449960,0.634805,0.458967,0.775178,0.675236,0.483397


In [54]:
#order column names by week again
import re

# Metadata columns 
metadata_cols = ['Env', 'Date_Planted', 'Date_Harvested']

# All other columns (the week-variable columns)
week_cols = [col for col in aggregate_weather_data_normalized.columns if col not in metadata_cols]

# Function to extract numeric week from column name
def extract_week_number(col_name):
    match = re.search(r'Week (-?\d+)', col_name)
    return int(match.group(1)) if match else float('inf')

# Sort week columns by extracted week number
sorted_week_cols = sorted(week_cols, key=extract_week_number)

# Reorder DataFrame
aggregate_weather_data_normalized = aggregate_weather_data_normalized[metadata_cols + sorted_week_cols]

In [55]:
aggregate_weather_data_normalized.head(5)

Week_Variable,Env,Date_Planted,Date_Harvested,ALLSKY_SFC_SW_DWN - Week -10,PRECTOTCORR - Week -10,RH2M - Week -10,T2MDEW - Week -10,T2M_DIFF_SUM - Week -10,T2M_GDD - Week -10,T2M_MAX - Week -10,...,WS2M - Week 30,ALLSKY_SFC_SW_DWN - Week 31,PRECTOTCORR - Week 31,RH2M - Week 31,T2MDEW - Week 31,T2M_DIFF_SUM - Week 31,T2M_GDD - Week 31,T2M_MAX - Week 31,T2M_MIN - Week 31,WS2M - Week 31
0,DEH1_2014,2014-05-05,2014-09-29,0.624749,0.056321,0.633801,0.368340,0.450691,0.444400,0.437943,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,GAH1_2014,2014-04-04,2014-09-11,0.441750,0.233222,0.709306,0.492940,0.503170,0.537441,0.542266,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,IAH1a_2014,2014-05-17,2014-10-21,0.779412,0.001365,0.508715,0.442350,0.699717,0.570127,0.624603,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,IAH1b_2014,2014-05-17,2014-10-20,0.779412,0.001365,0.508715,0.442350,0.699717,0.570127,0.624603,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,IAH1c_2014,2014-05-09,2014-10-20,0.574462,0.084109,0.716248,0.108263,0.604308,0.157490,0.197163,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
aggregate_weather_data_normalized.to_csv('aggregate_weather_data_normalized.csv', index=False)

In [58]:
len(aggregate_weather_data_normalized)

413

### Use substitute environments to aggregate weekly, apply padding and wide to long format for the missing TXH2 Envs
#### 11/April/2025

In [4]:
TXH2_data = pd.read_csv('final_unique_combinations_non_overlap_weather_data.csv')

In [5]:
TXH2_data
#5 missing envs

,Env,Date_Planted,Date_Harvested
0,TXH2_2015,2015-04-25,2015-09-22
1,TXH2_2016,2016-04-28,2016-09-22
2,TXH2_2016,2016-04-28,2016-09-23
3,TXH2_2017,2017-05-03,2017-09-23
4,TXH2_2017,2017-05-03,2017-10-09


In [6]:
TXH2_data.dtypes
#convert Date_Planted and Date_Harvested to datetime

Env               object
Date_Planted      object
Date_Harvested    object
dtype: object

In [7]:
TXH2_data['Date_Planted'] = pd.to_datetime(TXH2_data['Date_Planted'])
TXH2_data['Date_Harvested'] = pd.to_datetime(TXH2_data['Date_Harvested'])
TXH2_data.dtypes

Env                       object
Date_Planted      datetime64[ns]
Date_Harvested    datetime64[ns]
dtype: object

In [8]:
weather_extracted.head(5)

,Env,Date,T2MDEW,RH2M,WS2M,ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_MIN,PRECTOTCORR
0,DEH1_2015,2015-01-01,-3.86,74.81,3.13,10.75,6.00,-3.39,0.00
1,DEH1_2015,2015-01-02,-1.08,79.81,2.02,5.72,8.23,-1.47,0.00
2,DEH1_2015,2015-01-03,3.57,92.81,1.90,1.74,7.77,-1.35,6.70
3,DEH1_2015,2015-01-04,12.58,95.00,5.38,4.05,18.62,6.78,9.98
4,DEH1_2015,2015-01-05,-4.08,63.38,5.17,10.92,8.73,-3.14,0.00


In [9]:
weather_extracted.dtypes

Env                          object
Date                 datetime64[ns]
T2MDEW                      float64
RH2M                        float64
WS2M                        float64
ALLSKY_SFC_SW_DWN           float64
T2M_MAX                     float64
T2M_MIN                     float64
PRECTOTCORR                 float64
dtype: object

In [12]:
from datetime import timedelta

# Define substitute envs for TXH2 envs
txh2_substitutions = {
    'TXH2_2015': 'TXH1_2015',
    'TXH2_2016': 'TXH1_2016',
    'TXH2_2017': 'TXH1-Early_2017'
}

# Create an empty dictionary to store weather data for each unique (Env, Plant, Harvest) combo
weather_by_instance = {}

# Loop through each row in the TXH2-specific practise_envs dataframe
for _, row in TXH2_data.iterrows():
    env = row['Env']
    plant_date = pd.to_datetime(row['Date_Planted'])
    harvest_date = pd.to_datetime(row['Date_Harvested'])
    start_date = plant_date - timedelta(weeks=10)  # Start date is 10 weeks before Date_Planted

    # Use the substitute environment directly
    weather_env = txh2_substitutions[env]

    # Filter the weather_extracted DataFrame using the substitute environment
    filtered_weather = weather_extracted[
        (weather_extracted['Env'] == weather_env) &
        (weather_extracted['Date'] >= start_date) &
        (weather_extracted['Date'] < harvest_date)
    ].copy()

    # Store using the original TXH2 key
    key = (env, plant_date.strftime('%Y-%m-%d'), harvest_date.strftime('%Y-%m-%d'))
    weather_by_instance[key] = filtered_weather



In [14]:
for key, df in weather_by_instance.items():
    env, plant_str, harvest_str = key
    plant_date = pd.to_datetime(plant_str)
    
    # Calculate week number relative to Date_Planted
    df['Week'] = ((df['Date'] - plant_date).dt.days // 7)

In [15]:
weather_by_instance['TXH2_2016','2016-04-28','2016-09-23']

,Env,Date,T2MDEW,RH2M,WS2M,ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_MIN,PRECTOTCORR,Week
15029,TXH1_2016,2016-02-18,12.49,84.00,3.13,18.34,23.69,7.08,0.00,-10
15030,TXH1_2016,2016-02-19,15.41,84.44,3.24,17.07,24.62,13.54,0.00,-10
15031,TXH1_2016,2016-02-20,16.89,87.31,2.90,6.29,24.62,14.86,0.00,-10
15032,TXH1_2016,2016-02-21,17.28,91.69,1.72,5.70,22.12,15.85,8.77,-10
15033,TXH1_2016,2016-02-22,15.33,88.94,1.80,8.50,21.00,14.35,15.44,-10
...,...,...,...,...,...,...,...,...,...,...
15242,TXH1_2016,2016-09-18,24.21,79.19,1.25,20.89,32.89,24.10,0.01,20
15243,TXH1_2016,2016-09-19,23.68,76.00,1.34,21.70,34.29,24.19,0.01,20
15244,TXH1_2016,2016-09-20,23.44,76.31,1.07,20.06,34.34,23.44,0.43,20
15245,TXH1_2016,2016-09-21,20.67,69.56,0.97,22.23,34.24,22.46,0.03,20


In [16]:
#Weekly aggregation of weather variables

aggregated_weather_data = {}

for key, df in weather_by_instance.items():
    env, plant_str, harvest_str = key  # Extract env and dates
    plant_date = pd.to_datetime(plant_str)

    # Perform weekly aggregation
    aggregated_df = df.groupby('Week').agg(
        T2MDEW=('T2MDEW', 'sum'),
        RH2M=('RH2M', 'mean'),
        WS2M=('WS2M', 'sum'),
        ALLSKY_SFC_SW_DWN=('ALLSKY_SFC_SW_DWN', 'sum'),
        T2M_MAX=('T2M_MAX', 'mean'),
        T2M_DIFF_SUM=('T2M_MAX', lambda x: (x - df.loc[x.index, 'T2M_MIN']).sum()),
        T2M_GDD=('T2M_MAX', lambda x: (((x + df.loc[x.index, 'T2M_MIN']) / 2) - 10).sum()),
        T2M_MIN=('T2M_MIN', 'mean'),
        PRECTOTCORR=('PRECTOTCORR', 'sum')
    ).reset_index()

    # Store the result in aggregated_weather_data dictionary
    aggregated_weather_data[key] = aggregated_df

In [17]:
aggregated_weather_data['TXH2_2016','2016-04-28','2016-09-23']

,Week,T2MDEW,RH2M,WS2M,ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_DIFF_SUM,T2M_GDD,T2M_MIN,PRECTOTCORR
0,-10,90.40,83.670000,21.60,86.02,21.341429,72.68,43.050,10.958571,50.75
1,-9,65.92,76.702857,16.63,126.03,20.928571,98.43,27.285,6.867143,0.02
2,-8,101.20,83.045714,19.92,100.41,23.387143,80.34,53.540,11.910000,60.89
3,-7,111.56,83.498571,16.02,118.87,25.088571,80.33,65.455,13.612857,55.08
4,-6,77.97,78.670000,20.28,133.72,20.964286,83.08,35.210,9.095714,7.38
5,-5,91.05,81.258571,15.71,131.89,22.557143,87.02,44.390,10.125714,10.00
6,-4,75.81,71.724286,15.78,163.93,23.392857,97.32,45.090,9.490000,2.64
7,-3,102.96,75.518571,15.38,102.16,25.710000,79.34,70.300,14.375714,29.84
8,-2,129.04,86.562857,14.52,98.62,25.601429,62.33,78.045,16.697143,127.61
9,-1,127.35,81.687143,14.08,153.99,26.745714,67.90,83.270,17.045714,34.28


In [18]:
# padding
import numpy as np

# Define the full range of weeks (-10 to 31)
full_weeks = np.arange(-10, 32)  

# Loop through each dataframe in the dictionary
for key, df in aggregated_weather_data.items():
    
    # Create a dataframe with all weeks
    full_week_df = pd.DataFrame({'Week': full_weeks})
    
    # Merge with the existing dataframe on 'Week' column
    padded_df = full_week_df.merge(df, on='Week', how='left')
    
    # Fill missing values with 0
    padded_df.fillna(0, inplace=True)
    
    # Store the updated dataframe back in the aggregated_weather_data dictionary
    aggregated_weather_data[key] = padded_df

In [19]:
aggregated_weather_data['TXH2_2016','2016-04-28','2016-09-23'].tail(12)

,Week,T2MDEW,RH2M,WS2M,ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_DIFF_SUM,T2M_GDD,T2M_MIN,PRECTOTCORR
30,20,161.26,75.517143,7.47,147.31,33.69,71.6,130.03,23.461429,0.56
31,21,21.67,73.060000,1.60,19.18,34.15,11.9,18.20,22.250000,0.14
32,22,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.000000,0.00
33,23,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.000000,0.00
34,24,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.000000,0.00
35,25,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.000000,0.00
36,26,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.000000,0.00
37,27,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.000000,0.00
38,28,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.000000,0.00
39,29,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.000000,0.00


In [21]:
#rerun transpose_and_combine_data function up-top
transposed_combined_aggregated_TXH2_data = transpose_and_combine_data(aggregated_weather_data)

In [22]:
transposed_combined_aggregated_TXH2_data

Week_Variable,Env,Date_Planted,Date_Harvested,ALLSKY_SFC_SW_DWN - Week -1,ALLSKY_SFC_SW_DWN - Week -10,ALLSKY_SFC_SW_DWN - Week -2,ALLSKY_SFC_SW_DWN - Week -3,ALLSKY_SFC_SW_DWN - Week -4,ALLSKY_SFC_SW_DWN - Week -5,ALLSKY_SFC_SW_DWN - Week -6,...,WS2M - Week 29,WS2M - Week 3,WS2M - Week 30,WS2M - Week 31,WS2M - Week 4,WS2M - Week 5,WS2M - Week 6,WS2M - Week 7,WS2M - Week 8,WS2M - Week 9
0,TXH2_2015,2015-04-25,2015-09-22,128.93,90.12,109.62,98.44,124.44,132.04,94.07,...,0.0,14.94,0.0,0.0,16.98,7.85,10.55,19.80,10.57,12.08
1,TXH2_2016,2016-04-28,2016-09-22,153.99,86.02,98.62,102.16,163.93,131.89,133.72,...,0.0,14.08,0.0,0.0,15.62,11.70,13.98,10.47,9.75,18.58
2,TXH2_2016,2016-04-28,2016-09-23,153.99,86.02,98.62,102.16,163.93,131.89,133.72,...,0.0,14.08,0.0,0.0,15.62,11.70,13.98,10.47,9.75,18.58
3,TXH2_2017,2017-05-03,2017-09-23,171.37,117.23,151.70,125.68,138.54,150.17,135.12,...,0.0,17.83,0.0,0.0,9.56,15.42,18.97,14.86,18.52,12.26
4,TXH2_2017,2017-05-03,2017-10-09,171.37,117.23,151.70,125.68,138.54,150.17,135.12,...,0.0,17.83,0.0,0.0,9.56,15.42,18.97,14.86,18.52,12.26


In [24]:
#order column names by week
import re

# Metadata columns 
metadata_cols = ['Env', 'Date_Planted', 'Date_Harvested']

# All other columns (the week-variable columns)
week_cols = [col for col in transposed_combined_aggregated_TXH2_data.columns if col not in metadata_cols]

# Function to extract numeric week from column name
def extract_week_number(col_name):
    match = re.search(r'Week (-?\d+)', col_name)
    return int(match.group(1)) if match else float('inf')

# Sort week columns by extracted week number
sorted_week_cols = sorted(week_cols, key=extract_week_number)

# Reorder DataFrame
transposed_combined_aggregated_TXH2_data = transposed_combined_aggregated_TXH2_data[metadata_cols + sorted_week_cols]

In [25]:
transposed_combined_aggregated_TXH2_data

Week_Variable,Env,Date_Planted,Date_Harvested,ALLSKY_SFC_SW_DWN - Week -10,PRECTOTCORR - Week -10,RH2M - Week -10,T2MDEW - Week -10,T2M_DIFF_SUM - Week -10,T2M_GDD - Week -10,T2M_MAX - Week -10,...,WS2M - Week 30,ALLSKY_SFC_SW_DWN - Week 31,PRECTOTCORR - Week 31,RH2M - Week 31,T2MDEW - Week 31,T2M_DIFF_SUM - Week 31,T2M_GDD - Week 31,T2M_MAX - Week 31,T2M_MIN - Week 31,WS2M - Week 31
0,TXH2_2015,2015-04-25,2015-09-22,90.12,6.54,80.507143,51.07,95.12,6.150,17.672857,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TXH2_2016,2016-04-28,2016-09-22,86.02,50.75,83.670000,90.40,72.68,43.050,21.341429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,TXH2_2016,2016-04-28,2016-09-23,86.02,50.75,83.670000,90.40,72.68,43.050,21.341429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,TXH2_2017,2017-05-03,2017-09-23,117.23,0.47,78.652857,88.12,94.77,47.475,23.551429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TXH2_2017,2017-05-03,2017-10-09,117.23,0.47,78.652857,88.12,94.77,47.475,23.551429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
transposed_combined_aggregated_TXH2_data.to_csv('transposed_combined_aggregated_TXH2_data.csv', index=False)

### Combine TXH2 envs with other 413 envs

In [29]:
transposed_combined_agggregated_weather_data = pd.read_csv('transposed_combined_agggregated_weather_data.csv')

In [31]:
transposed_combined_agggregated_weather_data.head(5)

,Env,Date_Planted,Date_Harvested,ALLSKY_SFC_SW_DWN - Week -10,PRECTOTCORR - Week -10,RH2M - Week -10,T2MDEW - Week -10,T2M_DIFF_SUM - Week -10,T2M_GDD - Week -10,T2M_MAX - Week -10,...,WS2M - Week 30,ALLSKY_SFC_SW_DWN - Week 31,PRECTOTCORR - Week 31,RH2M - Week 31,T2MDEW - Week 31,T2M_DIFF_SUM - Week 31,T2M_GDD - Week 31,T2M_MAX - Week 31,T2M_MIN - Week 31,WS2M - Week 31
0,DEH1_2014,2014-05-05,2014-09-29,93.01,4.55,76.304286,-34.53,66.62,-74.950,4.051429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,GAH1_2014,2014-04-04,2014-09-11,67.50,18.81,80.685714,-5.32,73.49,-51.765,7.854286,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,IAH1a_2014,2014-05-17,2014-10-21,114.57,0.12,69.045714,-17.18,99.22,-43.620,10.855714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,IAH1b_2014,2014-05-17,2014-10-20,114.57,0.12,69.045714,-17.18,99.22,-43.620,10.855714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,IAH1c_2014,2014-05-09,2014-10-20,86.00,6.79,81.088571,-95.50,86.73,-146.445,-4.725714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
transposed_combined_agggregated_weather_data_TXH2_included = pd.concat([transposed_combined_agggregated_weather_data, 
                                                                        transposed_combined_aggregated_TXH2_data], 
                                                                       axis=0)


In [33]:
transposed_combined_agggregated_weather_data_TXH2_included.tail(6)

,Env,Date_Planted,Date_Harvested,ALLSKY_SFC_SW_DWN - Week -10,PRECTOTCORR - Week -10,RH2M - Week -10,T2MDEW - Week -10,T2M_DIFF_SUM - Week -10,T2M_GDD - Week -10,T2M_MAX - Week -10,...,WS2M - Week 30,ALLSKY_SFC_SW_DWN - Week 31,PRECTOTCORR - Week 31,RH2M - Week 31,T2MDEW - Week 31,T2M_DIFF_SUM - Week 31,T2M_GDD - Week 31,T2M_MAX - Week 31,T2M_MIN - Week 31,WS2M - Week 31
412,WIH3_2023,2023-04-26,2023-11-14,72.35,7.25,78.087143,-45.99,62.59,-94.155,1.020000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,TXH2_2015,2015-04-25,2015-09-22,90.12,6.54,80.507143,51.07,95.12,6.150,17.672857,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TXH2_2016,2016-04-28,2016-09-22,86.02,50.75,83.670000,90.40,72.68,43.050,21.341429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,TXH2_2016,2016-04-28,2016-09-23,86.02,50.75,83.670000,90.40,72.68,43.050,21.341429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,TXH2_2017,2017-05-03,2017-09-23,117.23,0.47,78.652857,88.12,94.77,47.475,23.551429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TXH2_2017,2017-05-03,2017-10-09,117.23,0.47,78.652857,88.12,94.77,47.475,23.551429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
len(transposed_combined_agggregated_weather_data_TXH2_included)

418

In [37]:
transposed_combined_agggregated_weather_data_TXH2_included.to_csv('transposed_combined_agggregated_weather_data_TXH2_included.csv', index=False)

### Normalise Data

In [7]:
import numpy as np
all_aggregated_weather_data = pd.read_csv('transposed_combined_agggregated_weather_data_TXH2_included.csv')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
#normalise weather variables

# Step 1: Keep first 3 columns as-is
all_aggregated_weather_data_normalized = all_aggregated_weather_data[['Env', 'Date_Planted', 'Date_Harvested']].copy()

# Step 2: Select columns to normalize
columns_to_normalize = all_aggregated_weather_data.columns.difference(['Env', 'Date_Planted', 'Date_Harvested'])

# Step 3: Normalize only non-zero values
scaler = MinMaxScaler()

for col in columns_to_normalize:
    values = all_aggregated_weather_data[col].values.astype(float)
    mask = values != 0  # mask out non-zero values

    if np.any(mask):  # only fit if there are non-zero values
        #reshape for MinMax package
        scaled = scaler.fit_transform(values[mask].reshape(-1, 1)).flatten()
        normalized_col = values.copy()
        normalized_col[mask] = scaled
        all_aggregated_weather_data_normalized[col] = normalized_col
    else:
        all_aggregated_weather_data_normalized[col] = values  # keep as is if all zero

In [8]:
#order column names by week again
import re

# Metadata columns 
metadata_cols = ['Env', 'Date_Planted', 'Date_Harvested']

# All other columns (the week-variable columns)
week_cols = [col for col in all_aggregated_weather_data_normalized.columns if col not in metadata_cols]

# Function to extract numeric week from column name
def extract_week_number(col_name):
    match = re.search(r'Week (-?\d+)', col_name)
    return int(match.group(1)) if match else float('inf')

# Sort week columns by extracted week number
sorted_week_cols = sorted(week_cols, key=extract_week_number)

# Reorder DataFrame
all_aggregated_weather_data_normalized = all_aggregated_weather_data_normalized[metadata_cols + sorted_week_cols]

In [17]:
all_aggregated_weather_data_normalized.head(5)

,Env,Date_Planted,Date_Harvested,ALLSKY_SFC_SW_DWN - Week -10,PRECTOTCORR - Week -10,RH2M - Week -10,T2MDEW - Week -10,T2M_DIFF_SUM - Week -10,T2M_GDD - Week -10,T2M_MAX - Week -10,...,WS2M - Week 30,ALLSKY_SFC_SW_DWN - Week 31,PRECTOTCORR - Week 31,RH2M - Week 31,T2MDEW - Week 31,T2M_DIFF_SUM - Week 31,T2M_GDD - Week 31,T2M_MAX - Week 31,T2M_MIN - Week 31,WS2M - Week 31
0,DEH1_2014,2014-05-05,2014-09-29,0.624749,0.056321,0.633801,0.368340,0.450691,0.444400,0.437943,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,GAH1_2014,2014-04-04,2014-09-11,0.441750,0.233222,0.709306,0.492940,0.503170,0.537441,0.542266,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,IAH1a_2014,2014-05-17,2014-10-21,0.779412,0.001365,0.508715,0.442350,0.699717,0.570127,0.624603,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,IAH1b_2014,2014-05-17,2014-10-20,0.779412,0.001365,0.508715,0.442350,0.699717,0.570127,0.624603,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,IAH1c_2014,2014-05-09,2014-10-20,0.574462,0.084109,0.716248,0.108263,0.604308,0.157490,0.197163,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
len(all_aggregated_weather_data_normalized)

418

In [18]:
all_aggregated_weather_data_normalized.to_csv('aggregate_weather_data_normalized_including_TXH2.csv', index=False)

### 🌱 Add Plant Growth Cycle
#### 11 April 2025

- Include the **Plant Growth Cycle** as an additional feature to help the machine learning model interpret time-based padding.  
- This feature captures the number of weeks between planting and harvest.  
- The goal is to help the model distinguish between padded values and biologically relevant data.
--------------
- The Plant Growth Cycle will be added to both datasets: one excluding TXH2 environments and one including them.  
- This allows us to assess whether model performance is heavily influenced by TXH2 data/ if TXH2 environments may be unreliable (see Weather_data_exploration.ipynb)

#### Dataset without TXH2 envs

In [26]:
aggregate_weather_data_normalized = pd.read_csv('aggregate_weather_data_normalized.csv')

In [27]:
len(aggregate_weather_data_normalized)

413

In [28]:
aggregate_weather_data_normalized['Date_Planted'] = pd.to_datetime(aggregate_weather_data_normalized['Date_Planted'])
aggregate_weather_data_normalized['Date_Harvested'] = pd.to_datetime(aggregate_weather_data_normalized['Date_Harvested'])
print(aggregate_weather_data_normalized.dtypes)

Env                                     object
Date_Planted                    datetime64[ns]
Date_Harvested                  datetime64[ns]
ALLSKY_SFC_SW_DWN - Week -10           float64
PRECTOTCORR - Week -10                 float64
                                     ...      
T2M_DIFF_SUM - Week 31                 float64
T2M_GDD - Week 31                      float64
T2M_MAX - Week 31                      float64
T2M_MIN - Week 31                      float64
WS2M - Week 31                         float64
Length: 381, dtype: object


In [29]:
# Calculate growth cycle in weeks and round 
aggregate_weather_data_normalized['Growth_Cycle_Weeks'] = (
    (aggregate_weather_data_normalized['Date_Harvested'] - aggregate_weather_data_normalized['Date_Planted'])
    .dt.days / 7
).round()

In [30]:
aggregate_weather_data_normalized.head()

,Env,Date_Planted,Date_Harvested,ALLSKY_SFC_SW_DWN - Week -10,PRECTOTCORR - Week -10,RH2M - Week -10,T2MDEW - Week -10,T2M_DIFF_SUM - Week -10,T2M_GDD - Week -10,T2M_MAX - Week -10,...,ALLSKY_SFC_SW_DWN - Week 31,PRECTOTCORR - Week 31,RH2M - Week 31,T2MDEW - Week 31,T2M_DIFF_SUM - Week 31,T2M_GDD - Week 31,T2M_MAX - Week 31,T2M_MIN - Week 31,WS2M - Week 31,Growth_Cycle_Weeks
0,DEH1_2014,2014-05-05,2014-09-29,0.624749,0.056321,0.633801,0.368340,0.450691,0.444400,0.437943,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0
1,GAH1_2014,2014-04-04,2014-09-11,0.441750,0.233222,0.709306,0.492940,0.503170,0.537441,0.542266,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0
2,IAH1a_2014,2014-05-17,2014-10-21,0.779412,0.001365,0.508715,0.442350,0.699717,0.570127,0.624603,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0
3,IAH1b_2014,2014-05-17,2014-10-20,0.779412,0.001365,0.508715,0.442350,0.699717,0.570127,0.624603,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0
4,IAH1c_2014,2014-05-09,2014-10-20,0.574462,0.084109,0.716248,0.108263,0.604308,0.157490,0.197163,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0


In [31]:
aggregate_weather_data_normalized.to_csv('aggregate_weather_data_normalized_with_growth_cycle.csv', index=False)

#### Dataset with TXH2 envs

In [19]:
aggregate_weather_data_normalized_including_TXH2 = pd.read_csv('aggregate_weather_data_normalized_including_TXH2.csv')

In [20]:
len(aggregate_weather_data_normalized_including_TXH2)

418

In [21]:
aggregate_weather_data_normalized_including_TXH2['Date_Planted'] = pd.to_datetime(aggregate_weather_data_normalized_including_TXH2['Date_Planted'])
aggregate_weather_data_normalized_including_TXH2['Date_Harvested'] = pd.to_datetime(aggregate_weather_data_normalized_including_TXH2['Date_Harvested'])
print(aggregate_weather_data_normalized_including_TXH2.dtypes)

Env                                     object
Date_Planted                    datetime64[ns]
Date_Harvested                  datetime64[ns]
ALLSKY_SFC_SW_DWN - Week -10           float64
PRECTOTCORR - Week -10                 float64
                                     ...      
T2M_DIFF_SUM - Week 31                 float64
T2M_GDD - Week 31                      float64
T2M_MAX - Week 31                      float64
T2M_MIN - Week 31                      float64
WS2M - Week 31                         float64
Length: 381, dtype: object


In [22]:
aggregate_weather_data_normalized_including_TXH2['Growth_Cycle_Weeks'] = (
    (aggregate_weather_data_normalized_including_TXH2['Date_Harvested'] - aggregate_weather_data_normalized_including_TXH2['Date_Planted'])
    .dt.days / 7
).round()

In [23]:
aggregate_weather_data_normalized_including_TXH2.head(5)

,Env,Date_Planted,Date_Harvested,ALLSKY_SFC_SW_DWN - Week -10,PRECTOTCORR - Week -10,RH2M - Week -10,T2MDEW - Week -10,T2M_DIFF_SUM - Week -10,T2M_GDD - Week -10,T2M_MAX - Week -10,...,ALLSKY_SFC_SW_DWN - Week 31,PRECTOTCORR - Week 31,RH2M - Week 31,T2MDEW - Week 31,T2M_DIFF_SUM - Week 31,T2M_GDD - Week 31,T2M_MAX - Week 31,T2M_MIN - Week 31,WS2M - Week 31,Growth_Cycle_Weeks
0,DEH1_2014,2014-05-05,2014-09-29,0.624749,0.056321,0.633801,0.368340,0.450691,0.444400,0.437943,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0
1,GAH1_2014,2014-04-04,2014-09-11,0.441750,0.233222,0.709306,0.492940,0.503170,0.537441,0.542266,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0
2,IAH1a_2014,2014-05-17,2014-10-21,0.779412,0.001365,0.508715,0.442350,0.699717,0.570127,0.624603,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0
3,IAH1b_2014,2014-05-17,2014-10-20,0.779412,0.001365,0.508715,0.442350,0.699717,0.570127,0.624603,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0
4,IAH1c_2014,2014-05-09,2014-10-20,0.574462,0.084109,0.716248,0.108263,0.604308,0.157490,0.197163,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0


In [25]:
aggregate_weather_data_normalized_including_TXH2.to_csv('aggregate_weather_data_normalized_including_TXH2_with_growth_cycle.csv', index=False)